In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

In [3]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
df = pd.read_csv("/content/drive/MyDrive/MercorTask/Training.csv")
df.head()

,user_id,question_id,answer_claude,justification_claude,score_claude,answer_openai,justification_openai,score_openai,answer_llama,justification_llama,...,alignment_with_oracle_llama,mae_with_oracle_llama,filled_prompt,answer_judge,justification_judge,score_judge,filled_prompt_x,filled_prompt_y,question_cand_answer,tokenized_questions
0,349,9,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate uses some technical terminology ...,2,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,...,True,0,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,"Interviewer: Certainly, I was asking about the...","[[101, 4357, 2121, 1024, 5121, 1010, 1045, 200..."
1,411,1,{'expertise': {'justification': 'The candidate...,The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate accurately uses technical termin...,4,"{'expertise': {'justification': ""The candidate...",The candidate struggles to accurately use tech...,...,False,2,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,Interviewer: Great! I see from your resume tha...,"[[101, 4357, 2121, 1024, 2307, 999, 1045, 2156..."
2,825,4,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate uses technical terminology such ...,4,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,...,False,1,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,"Interviewer: I'm sorry to interrupt, but could...","[[101, 4357, 2121, 1024, 1045, 1005, 1049, 337..."
3,765,7,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate uses technical terminology such ...,4,{'expertise': {'justification': 'The candidate...,The candidate demonstrates some technical know...,...,False,2,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,Interviewer: Great! Let's dive into the case s...,"[[101, 4357, 2121, 1024, 2307, 999, 2292, 1005..."
4,773,6,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a good understandin...,4,"{'expertise': {'justification': ""The candidate...",The candidate uses technical terminology such ...,4,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates some technical know...,...,False,2,\n\n### Evaluator Background\n\nYou are an exp...,"{'expertise': {'justification': ""The candidate...",The candidate demonstrates a solid understandi...,4,NaN,NaN,Interviewer: Your use of in-memory caching wit...,"[[101, 4357, 2121, 1024, 2115, 2224, 1997, 199..."


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
sw = stopwords.words('english')

def clean_text(text):

    text = text.lower()

    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs
    #text = re.sub(r"http", "",text)

    html=re.compile(r'<.*?>')

    text = html.sub(r'',text) #Removing html tags

    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations

    text = [word.lower() for word in text.split() if word.lower() not in sw]

    text = " ".join(text) #removing stopwords

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis

    return text

In [8]:
df['question_cand_answer'] = df['question_cand_answer'].apply(lambda x: clean_text(x))

In [9]:
questions = df.question_cand_answer.values
labels = df.score_judge.values

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
print(' Original: ', questions[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(questions[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(questions[0])))

 Original:  interviewer certainly, asking concept one hot encoding applied machine learning could walk process one hot encoding explain necessary certain machine learning tasks interviewee okay one hot encoding, refers strategy used machine learning data preprocessing categorical variables encoded binary vectors category represented unique binary value it, work that, first all, use categorical variables represent categorical colors like red, blue, green one hot encoding, category categorical variable assigned unique binary value example, lights let say taken example colors red, blue, green assign red , blue , grade may represented like commonly used ml algorithms, especially categorical variables inherent ordinal relationship allows algorithm effectively process categorical data numerical input machine learning require numerical input data benefit one hot encoding assuming ordinal relationship categories might exist, requires category treated distinct independent entity model training


In [12]:
max_len = 0

# For every sentence...
for question in questions:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(question, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (556 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  1508


In [13]:
input_ids = []
attention_masks = []

# For every tweet...
for question in questions:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        question,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', questions[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  interviewer certainly, asking concept one hot encoding applied machine learning could walk process one hot encoding explain necessary certain machine learning tasks interviewee okay one hot encoding, refers strategy used machine learning data preprocessing categorical variables encoded binary vectors category represented unique binary value it, work that, first all, use categorical variables represent categorical colors like red, blue, green one hot encoding, category categorical variable assigned unique binary value example, lights let say taken example colors red, blue, green assign red , blue , grade may represented like commonly used ml algorithms, especially categorical variables inherent ordinal relationship allows algorithm effectively process categorical data numerical input machine learning require numerical input data benefit one hot encoding assuming ordinal relationship categories might exist, requires category treated distinct independent entity model training
T

In [14]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
#val_size = int(0.2 * len(dataset))
val_size = len(dataset)  - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

4,819 training samples
1,205 validation samples


In [15]:
# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [16]:
# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# if device == "cuda:0":
# # Tell pytorch to run this model on the GPU.
#     model = model.cuda()
model = model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [19]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [20]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [21]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [26]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)-1

        unique_labels, counts = torch.unique(b_labels, return_counts=True)
        print(f"Unique labels: {unique_labels}")
        print(f"Counts: {counts}")

        # Ensure the sequences are truncated
        b_input_ids = b_input_ids[:, :512]
        b_input_mask = b_input_mask[:, :512]
        optimizer.zero_grad()
        try:
            output = model(b_input_ids,
                          token_type_ids=None,
                          attention_mask=b_input_mask,
                          labels=b_labels)
            loss = output.loss
            print("Step:", step, "Loss:", loss.item())  # Print loss for each step
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        except Exception as e:
            print("Error in step", step, ":", str(e))
            break

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)-1
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model = torch.load('bert_model')

In [ ]:
df_test = pd.read_csv('../input/nlp-getting-started/test.csv')
df_test['text'] = df_test['text'].apply(lambda x:clean_text(x))
test_tweets = df_test['text'].values